In [1]:
import os
os.chdir("../")
os.getcwd()

'/home/izam/coding/Loan-Prediction-System'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    target_column: str
    models_dir: str

In [3]:
from loanPrediction.constants import *
from loanPrediction.utils.common import read_yaml, create_directories, save_json

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            target_column = schema.name,
            models_dir = config.models_dir,
           
        )

        return model_evaluation_config

In [5]:
import os
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, auc
from sklearn.metrics import confusion_matrix
from loanPrediction.utils.common import load_bin, save_json, round_batch
from loanPrediction import logger
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    

    def _eval_metrics(self,actual, pred, dir):
        acc = accuracy_score(actual, pred)
        precision = precision_score(actual, pred)
        recall = recall_score(actual, pred)
        f1 = f1_score(actual, pred)
        rocauc_score = roc_auc_score(actual, pred)

        acc, precision, recall, f1, rocauc_score = round_batch(acc, precision, recall, f1, rocauc_score)

        metric = {
            "Accuracy" : acc,
            "Precision" : precision,
            "Recall": recall,
            "F1 score": f1,
            "roc auc score": rocauc_score,
        }
        logger.info(f"metrics are - {metric}")

        save_json(path=Path(os.path.join(dir, "metirc.json")), data=metric)
    
    
    def _eval_pics(self, actual, pred_proba, dir):
        logger.info(actual.shape)
        fpr, tpr, thresholds = roc_curve(actual, pred_proba)
        roc_auc = auc(fpr, tpr)

        # Plot ROC curve
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (AUC = {:.2f})'.format(roc_auc))
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Guessing')
        plt.xlabel('False Positive Rate (FPR)')
        plt.ylabel('True Positive Rate (TPR)')
        plt.title('Receiver Operating Characteristic (ROC) Curve')
        plt.legend(loc='lower right')

        plt.savefig(os.path.join(dir, "roc_curve.png"))
        plt.close()
        logger.info(f"Saved the ROC curve image at {os.path.join(dir, 'roc_curve.png')}")

        #confusion matrix
        y_pred = (pred_proba > 0.5).astype(int)
        cm = confusion_matrix(actual, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', annot_kws={"size": 16})
        plt.xlabel('Predicted Labels')
        plt.ylabel('True Labels')
        plt.title('Confusion Matrix')

        plt.savefig(os.path.join(dir, "confusion_matrix.png"))
        plt.close()
        logger.info(f"Saved the cunfusion matrix image at {os.path.join(dir, 'confusion_matrix.png')}")


    def evaluate(self):
        df = pd.read_csv(self.config.test_data_path)
        X = df.drop([self.config.target_column], axis=1)
        y = df[self.config.target_column]

        model = load_bin(path=Path(self.config.model_path))

        try:
            pred = model.predict(X)
            pred_proba = model.predict_proba(X)[:, 1]
        except Exception as e:
            logger.info("Can't predict with main model")
            raise e

        logger.info(f"predicted {pred.shape[0]} data points")
    
        self._eval_metrics(y, pred, self.config.root_dir)
        self._eval_pics(y, pred_proba, self.config.root_dir)

        models = os.listdir(self.config.models_dir)
        for model in models:
            model_path = os.path.join(self.config.models_dir, model)
            trained_model = load_bin(Path(model_path))

            model_name, _ = os.path.splitext(model)
            new_dir = os.path.join(self.config.root_dir, model_name)

            try:
                pred = trained_model.predict(X)
                pred_proba = trained_model.predict_proba(X)[:, 1]
            except Exception as e:
                logger.info(f"Can't predict with the model {model_name}")
                continue

            logger.info(f"predicted with {model}")
            

            create_directories([new_dir])

            self._eval_metrics(y, pred, new_dir)
            self._eval_pics(y, pred_proba, new_dir)
            



In [7]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.evaluate()
except Exception as e:
    raise e

[2023-12-11 18:48:34,577: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-11 18:48:34,583: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-11 18:48:34,586: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-11 18:48:34,587: INFO: common: created directory at: artifacts]
[2023-12-11 18:48:34,588: INFO: common: created directory at: artifacts/model_evaluation]
[2023-12-11 18:48:34,647: INFO: common: binary file loaded from: artifacts/model_trainer/model.joblib]
[2023-12-11 18:48:34,650: INFO: 2181041231: predicted 123 data points]
[2023-12-11 18:48:34,656: INFO: 2181041231: metrics are - {'Accuracy': 0.772, 'Precision': 0.759, 'Recall': 0.976, 'F1 score': 0.854, 'roc auc score': 0.655}]
[2023-12-11 18:48:34,657: INFO: common: json file saved at: artifacts/model_evaluation/metirc.json]
[2023-12-11 18:48:34,657: INFO: 2181041231: (123,)]
[2023-12-11 18:48:34,762: INFO: 2181041231: Saved the ROC curve image at artifacts/model_eval

In [8]:
import os
os.chdir("../")
os.listdir("./artifacts/model_trainer/models/")

FileNotFoundError: [Errno 2] No such file or directory: './artifacts/model_trainer/models/'